# Simple ideas of fusing modalities

- There are many ways to fuse modalities, in this we will start with a state-of-the-art approach where fusion happens in the intermediate layers
- An alternative to this - idea is feature/sensor concatenation, which is shown to be inferior by many research papers

# Getting Started-2
- Here we train the fushion classifier

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

2022-03-09 13:25:07.558713: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-03-09 13:25:08.846725: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-09 13:25:08.847424: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-03-09 13:25:08.898730: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.683GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-03-09 13:25:08.898765: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-03-09 13:25:08.901012: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.

In [3]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical

from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot

# Data loading

In [4]:
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values
 
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
        
    loaded = dstack(loaded)
    return loaded

In [5]:
# load train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

In [6]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [7]:
# Load the data
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [8]:
trainX_acc = trainX[:,:,:3]
print(trainX_acc.shape)
testX_acc = testX[:,:,:3]
print(testX_acc.shape)

trainX_gyro = trainX[:,:,6:]
print(trainX_gyro.shape)
testX_gyro = testX[:,:,6:]
print(testX_gyro.shape)

(7352, 128, 3)
(2947, 128, 3)
(7352, 128, 3)
(2947, 128, 3)


# Fushion modal model

In [9]:
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, MaxPooling1D, Flatten, Concatenate
from tcn import TCN

In [10]:
def get_model_tcn(n_timesteps, n_features_acc, n_features_gyro, nb_filters, n_outputs):
    
    i1 = keras.Input((n_timesteps, n_features_acc))
    i2 = keras.Input((n_timesteps, n_features_gyro))
    
    tcn1 = TCN(kernel_size=3,
            nb_filters=nb_filters,
            nb_stacks=3,
            use_skip_connections=False,
            use_batch_norm=False,
            use_weight_norm=False,
            use_layer_norm=False)(i1)
    
    tcn2 = TCN(kernel_size=3,
            nb_filters=nb_filters,
            nb_stacks=3,
            use_skip_connections=False,
            use_batch_norm=False,
            use_weight_norm=False,
            use_layer_norm=False)(i2)
    
    f1 = Flatten()(tcn1)
    f2 = Flatten()(tcn2)
    c1 = Concatenate(axis=1)([f1, f2])
    c1 = Dropout(0.2)(c1)
    d1 = Dense(64, activation='relu')(c1)
    output = Dense(n_outputs, activation='softmax')(d1)
    model = Model(inputs=[i1, i2], outputs=output, name="fushion_model")
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    return model

In [17]:
verbose, epochs, batch_size = 1, 50, 128
n_timesteps, n_features, n_outputs = trainX_acc.shape[1], trainX_acc.shape[2], trainy.shape[1]

model= get_model_tcn(n_timesteps, n_features, n_features, 16, n_outputs)

In [12]:
model.summary()

Model: "fushion_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 3)]     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128, 3)]     0                                            
__________________________________________________________________________________________________
tcn (TCN)                       (None, 16)           27664       input_1[0][0]                    
__________________________________________________________________________________________________
tcn_1 (TCN)                     (None, 16)           27664       input_2[0][0]                    
______________________________________________________________________________________

In [13]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [14]:
model.fit([trainX_acc, trainX_gyro], trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/50


/home/nesl/anaconda3/envs/tf25/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:3503: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(
2022-03-09 13:25:13.986055: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-09 13:25:14.004102: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3497870000 Hz
2022-03-09 13:25:14.029763: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2022-03-09 13:25:14.765049: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-03-09 13:25:14.815565: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output:

58/58 [==============================] - 44s 547ms/step - loss: 370.5094 - accuracy: 0.2061
Epoch 2/50
58/58 [==============================] - 20s 339ms/step - loss: 2.7876 - accuracy: 0.4331
Epoch 3/50
58/58 [==============================] - 19s 332ms/step - loss: 1.6661 - accuracy: 0.5437
Epoch 4/50
58/58 [==============================] - 20s 338ms/step - loss: 1.2342 - accuracy: 0.6025
Epoch 5/50
58/58 [==============================] - 19s 332ms/step - loss: 0.9700 - accuracy: 0.6543
Epoch 6/50
58/58 [==============================] - 19s 334ms/step - loss: 0.8289 - accuracy: 0.6898
Epoch 7/50
58/58 [==============================] - 19s 331ms/step - loss: 0.6892 - accuracy: 0.7212
Epoch 8/50
58/58 [==============================] - 19s 332ms/step - loss: 0.6399 - accuracy: 0.7457
Epoch 9/50
58/58 [==============================] - 19s 329ms/step - loss: 0.5702 - accuracy: 0.7716
Epoch 10/50
58/58 [==============================] - 19s 332ms/step - loss: 0.4940 - accuracy: 0.794

In [16]:
# evaluate model
_, accuracy = model.evaluate([testX_acc, testX_gyro], testy, batch_size=batch_size, verbose=0)

print('Test accuracy of model is:', accuracy)

Test accuracy of model is: 0.9222938418388367
